<a href="https://colab.research.google.com/github/jessica-aaao/ChordsExtractor/blob/main/ChordExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [30]:
!python3 -m pip install -q -U "yt-dlp[default]"
!pip install -q -U openai-whisper
!pip install -q -U demucs

In [31]:
import requests
import json
import pandas as pd
import os
import re
import unicodedata

In [32]:
from google.colab import drive
from IPython.display import display
from bs4 import BeautifulSoup

In [33]:
!git clone  https://github.com/mikezzb/lyrics-sync.git
!git clone https://github.com/filipecalegario/ISMIR2019-Large-Vocabulary-Chord-Recognition.git

fatal: destination path 'lyrics-sync' already exists and is not an empty directory.
fatal: destination path 'ISMIR2019-Large-Vocabulary-Chord-Recognition' already exists and is not an empty directory.


In [34]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Common

In [35]:
class SongUrls:
    def __init__(self, name, audio, lyrics, chords):
        self.name = name
        self.audio = audio
        self.lyrics = lyrics
        self.chords = chords

    def get_name(self):
        return self.name

    def get_audio_url(self):
        return self.audio

    def get_lyrics_url(self):
        return self.lyrics

    def get_chords_url(self):
        return self.chords

In [36]:
def get_urls():
    print(f'Fetching urls...\n\n')

    file_path = '/content/drive/My Drive/TCC/CodeData/songs.csv'
    songs = pd.read_csv(file_path)

    print(f'Urls fetched!\n\n')

    return songs

In [37]:
def get_songs_from_csv():
    songs = get_urls()

    song_urls = []

    print(f'Creating SongUrls...\n\n')

    for index, row in songs.iterrows():
        song_name = slugify(row['Song Name'])
        audio_url = row['Audio URL']
        lyrics_url = row['Lyrics URL']
        chords_url = row['Chords URL']

        song_urls.append(SongUrls(song_name, audio_url, lyrics_url, chords_url))

    print(f'SongUrls created!\n\n')

    return song_urls


In [38]:
def slugify(raw_song_name):
    song_name = raw_song_name.lower()

    song_name = unicodedata.normalize('NFKD', song_name)
    song_name = song_name.encode('ascii', 'ignore').decode('ascii')

    song_name = re.sub(r'[^a-z0-9]+', '_', song_name)

    song_name = song_name.strip('_')

    return song_name

# Sound

In [39]:
def extract_voice_from_audio(audio_path, song_name):
    print(f'Extracting vocals from {audio_path}...\n\n')

    !demucs --two-stems=vocals {audio_path} -o vocals/

    return f"/content/vocals/htdemucs/{song_name}/vocals.wav"


In [40]:
def extract_audio_from_video(youtube_url, song_name):
    cookies_path = '/content/cookies.txt'
    output_path = f"/content/audios/{song_name}.wav"

    !yt-dlp {youtube_url} --audio-format "wav" --cookies {cookies_path} -x -o {output_path}  -q

    return f"{output_path}"

In [41]:
def extract_sound_recording(youtube_url, song_name):
    output_path = extract_audio_from_video(youtube_url, song_name)

    print(f'Audio saved as {output_path}!\n\n')

    return output_path

# Chords

## Chords Extraction from Audio

In [42]:
def extract_chords_from(song_path, song_name):
    """Extrai a cifra, com o timestamp, a partir da música"""
    print(f"Extracting chords from {song_path}...\n\n")

    output_folder = "/content/chords"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_path = f"{output_folder}/{song_name}.lab"

    %cd ISMIR2019-Large-Vocabulary-Chord-Recognition
    !pip install -q -r requirements.txt
    !python chord_recognition.py {song_path} {output_path}
    %cd ..
    return output_path

## Chords Sync to Lyrics

### Chords Parsing

In [43]:
def simplify_chord(chord):
    chord = chord.replace(":", "")
    chord = chord.replace("min", "m")
    chord = chord.replace("maj", "")
    chord = chord.replace("hdim7", "m7(b5)")
    chord = chord.replace("hdim", "m7(b5)")
    chord = chord.replace("sus4(b7)", "7sus4")

    return chord

In [44]:
def clean_chords(chords):
    """
    Dada uma lista de acordes (ordenados por tempo),
    remove repetições imediatas do mesmo acorde.
    """
    if not chords:
        return []
    result = [chords[0]]

    for i in range(1, len(chords)):
        current = chords[i]
        previous = result[-1]
        if current["chord"] != previous["chord"]:
            result.append(current)
        else:
            previous["end"] = current["end"]

    return result

In [45]:
def parse_chords(chords_path):
    # Ler ACORDES do arquivo LAB
    print(f"Parsing chords from {chords_path}...")

    chords = []
    with open(chords_path, 'r', encoding='utf-8') as file:
        for line in file:
            start, end, chord = line.strip().split('\t')
            if chord == 'N':
                continue
            chord = simplify_chord(chord)
            chords.append({
                "start": float(start),
                "end": float(end),
                "chord": chord
            })

        print(f"Chords parsed!\n\n")
        print(chords)
        return clean_chords(chords)

### Lyrics Parsing

In [46]:
def parse_lyrics(timestamped_lyrics_path, lyrics_path):
    # Ler LETRA_TRANSCRITA do arquivo CSV
    print(f"Parsing lyrics from {lyrics_path}...")

    timestamped_lyrics = pd.read_csv(timestamped_lyrics_path)
    timestamped_per_line = []

    with open(lyrics_path, 'r', encoding='utf-8') as file:
        for index, line in enumerate(file):
            line = line.strip().split()
            words_in_line = len(line)

            timestamped_words = timestamped_lyrics.iloc[:words_in_line]
            timestamped_lyrics = timestamped_lyrics.iloc[words_in_line:]
            timestamped_per_line.append([timestamped_words])

    print(f'Lyrics Parsed!\n\n')

    return timestamped_per_line

### General

In [47]:
def overlay_chords_on_transcribed(lyrics_per_line, chords):
    """
    Associa acordes às palavras da leytra com base nos tempos de ACORDES.
    """
    print(f"Overlaying chords on lyrics...")

    result = []
    previous_end = 0
    for line_number, line in enumerate(lyrics_per_line):
        for df in line:
            for index, row in df.iterrows():
                start = row["start"]
                end = row["end"]
                word = row["label"]
                chord_start = None
                chord_name = None
                chord_end = None

                if previous_end is not None and start is not None and end is not None:
                    for chord_infos in chords:
                        chord_info_start = chord_infos["start"]
                        chord_info_end = chord_infos["end"]
                        chord_info_name = chord_infos["chord"]

                        if  start <= chord_info_start <= end:
                            chord_name = chord_info_name
                            chord_start = chord_info_start
                            chord_end = chord_info_end
                        elif previous_end <= chord_info_start < start:
                            word = f' {word}'
                            chord_name = chord_info_name
                            chord_start = chord_info_start
                            chord_end = chord_info_end

                result.append({
                    "word": word,
                    "start": start,
                    "end": end,
                    "chord": chord_name,
                    "chord_start": chord_start,
                    "chord_end": chord_end,
                    "line": line_number
                })
                previous_end = end

    print(f"Chords overlayed!\n\n")

    return result

In [62]:
def align_chord_over_word(word_info):
    word = word_info["word"]
    word_start = word_info["start"]
    word_end = word_info["end"]
    chord = word_info["chord"]
    chord_start = word_info["chord_start"]
    chord_end = word_info["chord_end"]

    word_duration = word_end - word_start
    ratio = (chord_start - word_start) / word_duration

    word_len = len(word)
    word_index = int(round(ratio * (word_len - 1)))
    word_index = max(0, min(word_index, word_len - 1))

    left_padding = " " * word_index
    chord_with_padding = left_padding + chord

    if len(chord_with_padding) < word_len:
        chord_with_padding += " " * (word_len - len(chord_with_padding))

    return chord_with_padding

In [49]:
def format_transcribed_with_chords(lyrics_with_chords):
    """
    Gera o formato de saída com acordes acima das palavras.
    """
    formatted_output = []
    chord_line = []
    lyrics_line = []

    current_line = lyrics_with_chords[0]["line"]
    for word_info in lyrics_with_chords:
        if word_info["line"] != current_line:
            current_line = word_info["line"]
            formatted_output.append(" ".join(chord_line))
            formatted_output.append(" ".join(lyrics_line))
            formatted_output.append("")
            chord_line = []
            lyrics_line = []

        word = word_info["word"]
        chord = word_info["chord"]

        if chord:
            if word == " ":
                chord_line.append(chord)
            else:
                chord_line.append(align_chord_over_word(word_info))
        else:
            chord_line.append(" " * len(word))

        lyrics_line.append(word)

    # Adiciona o restante, se existir
    if chord_line or lyrics_line:
        formatted_output.append(" ".join(chord_line))
        formatted_output.append(" ".join(lyrics_line))

    return "\n".join(formatted_output)

In [50]:
def format_song_with_chords(timestamped_lyrics_path, lyrics_path, chords_path):
    """
    Formata a LETRA_TRANSCRITA com acordes no formato tradicional de cifras.
    """
    lyrics = parse_lyrics(timestamped_lyrics_path, lyrics_path)
    chords = parse_chords(chords_path)

    lyrics_with_chords = overlay_chords_on_transcribed(lyrics, chords)

    # Formatar saída com acordes e palavras
    formatted_output = format_transcribed_with_chords(lyrics_with_chords)
    print(formatted_output)
    return formatted_output



---



# Lyrics

## Lyrics Extraction from Webpage

In [51]:
def save_to_file(data, song_name):
    folder_path = "/content/lyrics"
    path = f"{folder_path}/{song_name}.txt"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(path, 'w') as file:
        file.write(data)

    print(f'Saved as {path}')

    return path

In [52]:
def extract_lyrics_from_html(html):
    """Extrai a letra da página HTML fornecida"""

    print(f'Fetching lyrics...!\n\n')

    lyricsTag = html.find('div', class_='lyric-original')
    lyrics = ""

    for p in lyricsTag.find_all('p'):
        for br in p.find_all('br'):
            br.replace_with('\n')
        lyrics += p.get_text() + "\n"

    print(f'Lyrics fetched!\n\n')

    return lyrics


In [53]:
def get_lyrics_from_webpage(lyric_url, song_name):
    """Obtém a página web e extrai a letra"""

    print(f'Fetching webpage {lyric_url}...\n\n')
    response = requests.get(lyric_url)

    if response.status_code == 200:
        print(f'Webpage fetched!\n\n')
        htmlContent = BeautifulSoup(response.content, 'html.parser')

        lyrics = extract_lyrics_from_html(htmlContent)
        lyrics_path = save_to_file(lyrics, song_name)

        return lyrics_path
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}\n\n")

## Lyrics Sync to Audio

### Transcription + Manual Adjustment Attempt

In [54]:
def adjust_lyrics(transcribed_segments, comparison_lyrics):
    """Compara a letra transcrita e a letra extraída da Web, corrigindo a letra transcrita"""

    print(f"Adjusting lyrics...\n\n")

    for index, segment in enumerate(transcribed_segments):
        try:
            line = comparison_lyrics[index]

            if segment['text'].lower() != line.lower():
                segment['text'] = line

            words = line.split()
            # Testar com músicas não "especiais", mais enxutas
            if len(words) != len(segment['words']):
                print(f"Word count mismatch: {len(words)} != {len(segment['words'])}")
                continue

            for i, word in enumerate(words):
                try:
                    transcribed_word = segment['words'][i]['word']

                    if transcribed_word.lower().strip() != word.lower():
                        print(f"Word mismatch: {transcribed_word} != {word}")

                        segment['words'][i]['word'] = word
                except:
                    print(f"Word not found: {word}")
                    continue
        except:
            print(f"Line not found: {segment['text']}")
            continue

    print(f"Lyrics adjusted:\n{json.dumps(transcribed_lyrics, indent=2)}\n\n")

    return transcribed_lyrics

In [55]:
def transcribe_audio(audio_path):
    """Transcreve o áudio usando o modelo Whisper"""

    print(f"Transcribing audio...\n\n")

    model = whisper.load_model("turbo", device='cuda')
    result = model.transcribe(audio_path, word_timestamps=True)

    print(f"Audio Transcribed!\n\n")

    return result['segments']


### Using Lyrics-Sync

In [56]:
def create_output_folder():
    output_folder = "/content/lyrics-sync/output"
    vocals_folder = output_folder + "/vocals"
    words_folder = output_folder + "/words"
    lrc_folder = output_folder + "/lrc"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(vocals_folder):
        os.makedirs(vocals_folder)

    if not os.path.exists(words_folder):
        os.makedirs(words_folder)

    if not os.path.exists(lrc_folder):
        os.makedirs(lrc_folder)

In [57]:
def get_timestamps(audio_path, lyrics_path, song_name):
    print("Installing conda...")
    !wget -c https://repo.continuum.io/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
    !chmod +x Anaconda3-2024.10-1-Linux-x86_64.sh
    !bash ./Anaconda3-2024.10-1-Linux-x86_64.sh -b -f -p /usr/local
    print("Conda installed!")

    print("Installing lsync...")
    %cd /content/lyrics-sync/
    !conda env update -f environment.yml
    !source activate lsync

    print("Lsync installed!")
    from lsync import LyricsSync

    print("Extracting timestamps...")
    lsync = LyricsSync()
    words, lrc = lsync.sync(audio_path, lyrics_path)
    print("Timestamps extracted!")

    return f"/content/lyrics-sync/output/words/{song_name}.csv"
    %cd ..

In [58]:
def get_synced_lyrics(lyric_url, audio_path, song_name):
    """Obtém a letra sincronizada com o áudio e corrigida"""

    print(f"Getting synced lyrics...\n\n")

    lyrics_path = get_lyrics_from_webpage(lyric_url, song_name)
    timestamps_path = get_timestamps(audio_path, lyrics_path, song_name)

    print(f"Synced lyrics ready!\n\n")

    return timestamps_path

# MAIN

In [ ]:
# Extrai dados do csv e converte pra [SongUrls]
songs = get_songs_from_csv()

song = songs[2]
song_name = song.get_name().replace(" ", "_")
print(f"Generating \'{song_name}\' chord sheet...")

Fetching urls...


Urls fetched!


Creating SongUrls...


SongUrls created!


Generating 'petrolina_juazeiro' chord sheet...


In [ ]:
# Baixa músicas do youtube
song_path = extract_sound_recording(song.get_audio_url(), song_name)

In [ ]:
chords_path = extract_chords_from(song_path, song_name)

In [ ]:
# Extrai letras com timestamp
create_output_folder()
lyrics_timestamped_path = get_synced_lyrics(song.get_lyrics_url(), song_path, song_name)

In [61]:
raw_chords = format_song_with_chords("/content/petrolina_juazeiro.csv", "/content/Petrolina_Juazeiro.txt", "/content/petrolina_juazeiro.lab")

Parsing lyrics from /content/Petrolina_Juazeiro.txt...
Lyrics Parsed!


Parsing chords from /content/petrolina_juazeiro.lab...
Chords parsed!


[{'start': 0.023219954648526078, 'end': 2.2291156462585033, 'chord': 'Dm'}, {'start': 2.2291156462585033, 'end': 3.9938321995464854, 'chord': 'Am'}, {'start': 3.9938321995464854, 'end': 5.6656689342403626, 'chord': 'E7'}, {'start': 5.6656689342403626, 'end': 7.4071655328798185, 'chord': 'Am'}, {'start': 7.4071655328798185, 'end': 9.148662131519275, 'chord': 'Dm'}, {'start': 9.148662131519275, 'end': 10.936598639455783, 'chord': 'Am'}, {'start': 10.936598639455783, 'end': 12.60843537414966, 'chord': 'E7'}, {'start': 12.60843537414966, 'end': 14.396371882086168, 'chord': 'Am'}, {'start': 14.396371882086168, 'end': 15.27873015873016, 'chord': 'D/3'}, {'start': 15.27873015873016, 'end': 16.137868480725626, 'chord': 'E/3'}, {'start': 16.137868480725626, 'end': 17.740045351473924, 'chord': 'Am'}, {'start': 17.740045351473924, 'end': 19.57442176870748